In [ ]:
from __future__ import division, print_function

# Third-party
from astropy.constants import G
import astropy.cosmology as ac
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
cosmo.lookback_time(2.5)

In [ ]:
M_vir0 = 1E12 * u.Msun
cosmo = ac.Planck15

In [ ]:
def Delta(z): 
    """ An approximation thanks to Dekel & Birnboim 2006 (see appendix) """
    return (18*np.pi**2 - 82*cosmo.Ode(z) - 39*cosmo.Ode(z)**2) / cosmo.Om(z)

def M_vir(z):
    return M_vir0 * np.exp(-0.9*z)

def R_vir(z):
    _Delta = (Delta(z) / 200)**(-1/3.)
    _Om = (cosmo.Om(z) / 0.3)**(-1/3.)
    _h2 = (cosmo.h / 0.7)**(-2/3.)
    return 309 * (M_vir(z)/(1E12*u.Msun))**(1/3) * _Delta * _Om * _h2 / (1 + z) * u.kpc

In [ ]:
def f_star(z, M_star0):
    """ Code from Sam Leitner: https://github.com/sleitner/star-formation-histories"""
    R = 0.45
    beta = -0.35
    alpha = 3.45
    M0 = 10**11.0
    dadt = 0.064/1e9
    A0 = 3.24
    r11 = (M_star0/M0)**(1+beta)
    return M0/M_star0*( ((M_star0/M0)**-beta + beta/dadt * (1-R)/(alpha-1)*A0/M0*((1+z)**(alpha-1)-1)))**(-1/beta)

def r_star(z, r_star0):
    return r_star0 * R_vir(z) / R_vir(0.)

In [ ]:
# TODO: get all of this from importing from uncluster.potential
M_disk0 = 6.8E10 * u.Msun
M_bulge0 = 5E9 * u.Msun
M_nucl0 = 1.76E9 * u.Msun

r_disk0 = 3. * u.kpc
r_bulge0 = 1 * u.kpc
r_nucl0 = 67.9 * u.pc
r_halo = 16.*u.kpc

def get_potential(z):
    
    Mvir = M_vir(z)
    rvir = R_vir(z)
    
    c = rvir / r_halo
    M_halo = Mvir / (np.log(c+1) - c/(c+1))
    
    M_disk = M_disk0 * f_star(z, M_disk0.to(u.Msun).value)
    M_bulge = M_bulge0 * f_star(z, M_bulge0.to(u.Msun).value)
    M_nucl = M_nucl0 * f_star(z, M_nucl0.to(u.Msun).value)
    
    if np.isnan(M_bulge) or M_bulge < 0.:
        M_bulge = 0.
    
    if np.isnan(M_nucl) or M_nucl < 0.:
        M_nucl = 0.
    
    rs_disk = r_star(z, r_disk0)
    rs_bulge = r_star(z, r_bulge0)
    rs_nucl = r_star(z, r_nucl0)
    
    mw_potential = gp.CCompositePotential()
    mw_potential['nucl'] =  gp.HernquistPotential(m=M_nucl, c=rs_nucl, units=galactic)
    mw_potential['bulge'] = gp.HernquistPotential(m=M_bulge, c=rs_bulge, units=galactic)
    mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=M_disk, a=rs_disk, b=280*u.pc, 
                                                     units=galactic)

    # for DM halo potential
    v_c = np.sqrt(((np.log(2.) - 0.5) * (G * M_halo / r_halo)).decompose(galactic).value)
    mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=r_halo, units=galactic)
    
    return mw_potential

In [ ]:
z = np.linspace(0,5,256)
t = cosmo.lookback_time(z)

fig,axes = plt.subplots(1,2,figsize=(10,4), sharey=True)

for x in range(8,11+1):
    axes[0].plot(t, f_star(z, 10**x), marker='', label=r'$10^{{{}}} \, {{\rm M}}_\odot$'.format(x))
    axes[1].plot(z, f_star(z, 10**x), marker='')
    
axes[0].set_xlim(0, 13.7)
axes[1].set_xlim(5, 0)
axes[0].set_xlabel('lookback time [Gyr]')
axes[0].set_ylabel("$f_*(t)$")
axes[1].set_xlabel('redshift, $z$')

axes[0].legend()

fig.tight_layout()

In [ ]:
zz = np.linspace(0, 2.5, 128)

M_enc = dict()
M_enc['rs_h'] = []
for z in zz:
    pot = get_potential(z)
    M_enc['rs_h'].append(pot.mass_enclosed([r_halo.value,0.,0]*u.kpc)[0])
        
for k in M_enc.keys():
    M_enc[k] = u.Quantity(M_enc[k])

In [ ]:
style = dict(marker='', linewidth=2.)

fig,axes = plt.subplots(1, 2, figsize=(8,4), sharex=True)

axes[0].semilogy(zz, M_vir(zz), label=r'$M_{\rm vir}$', **style)
axes[0].semilogy(zz, M_enc['rs_h'], label=r'$M(<r_{\rm H})$', marker='', linewidth=1.)

axes[1].semilogy(zz, R_vir(zz), **style)

axes[0].set_xlim(zz.max(), zz.min())
axes[0].set_ylim(1E10, 3E12)

axes[0].set_xlabel('redshift, $z$')
axes[1].set_xlabel('redshift, $z$')

axes[0].set_ylabel(r'$M_{\rm vir}\,[{\rm M}_\odot]$')
axes[1].set_ylabel(r'$r_{\rm vir}\,[{\rm kpc}]$')

axes[0].legend(loc='upper left')

fig.suptitle("Evolution of Milky Way mass model", fontsize=20, y=0.98)#, x=0.55)

fig.tight_layout()
fig.subplots_adjust(top=0.86)
fig.savefig("../paper/figures/mass-evolution.pdf")